In [53]:
from rnn_data import load_imdb
import numpy as np

In [54]:
(x_train, y_train), (x_val, y_val), (i2w, w2i), numcls = load_imdb(final=False)

In [55]:
# Remove the 10 longest reviews to have shorter padding
for i in range(10):
    max_len_idx = np.argmax([len(i) for i in x_train])
    del x_train[max_len_idx]
    del y_train[max_len_idx]


1853
1773
1646
1587
1568
1555
1427
1422
1392
1341


In [56]:
# Pad the sentences to the longest sentence in the training data
def pad(l, length): 
    pad_idx = w2i['.pad']
    return [i + [pad_idx] * (length - len(i)) for i in l]

max_len = np.max([len(i) for i in x_train])
x_train_pad = pad(x_train, max_len)
x_val_pad = pad(x_val, max_len)

In [57]:
# Turn data into batches of all size 4
def make_batch(l): return [l[i:i+4 % len(l)] for i in range(0, len(l), 4)]

x_batches = make_batch(x_train_pad)
y_batches = make_batch(y_train)
x_batches_val = make_batch(x_val_pad)
y_batches_val = make_batch(y_val)


In [58]:

print(x_batches[5])

[[11, 9, 38, 220, 3946, 731, 2490, 207, 69, 20, 34, 2168, 22, 4, 605, 2176, 320, 9, 27, 101, 53, 386, 4, 320, 85, 33, 131, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0